In [1]:
from tensorflow.keras.layers import Dense, LSTM, Conv2D, TimeDistributed, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [2]:
frames = 100
x = 160
y = 120
classes = ["Reading", "Drink", "Eat", "Call Cellphone", "Write", "Use Laptop", "Use Vaccum Cleaner", "Cheer Up", "Sit Still", "Toss Paper", "Play Game", "Lay Down", "Walk", "Stand Up", "Sit Down"]
outputs = 2


In [3]:
model = Sequential()
model.add(TimeDistributed(Conv2D(input_shape=(frames, x, y, 1), filters=64,kernel_size=(3,3),padding="same", activation="relu")))
model.add(TimeDistributed(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")))
model.add(TimeDistributed(GlobalAveragePooling2D()))
model.add(LSTM(frames, input_shape=(frames,1), activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(10)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(outputs, activation='softmax'))

In [4]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.build((10,frames,x,y,1))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (10, 100, 160, 120, 64)  640       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (10, 100, 160, 120, 64)  36928     
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (10, 100, 64)            0         
 tributed)                                                       
                                                                 
 lstm (LSTM)                 (10, 100, 100)            66000     
                                                                 
 time_distributed_3 (TimeDis  (10, 100, 10)            1010      
 tributed)                                              

In [5]:
import cv2 as cv
import os

# for f in os.listdir("MSRDailyAct3D_pack2/Reading"):
#     if f.endswith(".avi"):
#         continue
#     os.remove(os.path.join("MSRDailyAct3D_pack2/Reading", f))

i=0
c=0
X_train = []
Y_train = []
no_act = 2
no_video_frames = 5
for path in os.listdir("SRI Datasets/MSR_U"):
    
    if no_act == 0:
        break
    
    no_video_frames = 5
    for path1 in os.listdir("SRI Datasets/MSR_U/"+path):
        
        if no_video_frames == 0:
            break
    
        capture = cv.VideoCapture("SRI Datasets/MSR_U/"+path+"/"+path1)

        #work

        i = i+1   

        temp_frames = []
        for i in range(frames):

            ret, frame = capture.read()
            #print(ret)
            if ret == True:
                
                scale_precent = 25
                width = int(frame.shape[1] * scale_precent / 100)
                height = int(frame.shape[0] * scale_precent / 100)
#                 print(width)
#                 print(height)
                dim = (height,width)

                resized = cv.resize(frame, dim, interpolation = cv.INTER_AREA)
                grayImage = cv.cvtColor(resized, cv.COLOR_BGR2GRAY)
#                 cv.imshow('Frame '+ str(i), grayImage)
                temp_frames.append(grayImage)


            # Break the loop
            else: 
                break

        if len(temp_frames) < frames:
            for t in range(frames-len(temp_frames)):
                temp_frames.append(np.zeros((x,y)))
        X = np.array(temp_frames)
        X=X.reshape(X.shape[0],X.shape[1],X.shape[2],1)
        X=X.astype("float16")
        ind = 0
        label = np.zeros((X.shape[0],outputs))
        label[:][ind] = 1
        Y_train.append(label)
        #print(label.shape)
        X_train.append(X)
        no_video_frames = no_video_frames - 1

    no_act = no_act-1
    
    
capture.release()
cv.destroyAllWindows()

In [6]:
X_train_final = np.array(X_train)
Y_train_final = np.array(Y_train)

#model.fit(X_train_final, Y_train_final)

In [ ]:
model.fit(X_train_final, Y_train_final, epochs=1)

In [ ]:
model.save('HAR_weights.h5')